# 🖼️ Parallel Image Captioning with the Image Captioning Server

This notebook demonstrates how to **dynamically scan images** in a directory and send **parallel POST requests** to the `/caption` endpoint of the **Image Captioning Server**.  
We use **aiohttp** and **asyncio** for concurrency, and dynamically load images using Python's `glob` module.

The server is running at **http://localhost:80**.


In [ ]:
import aiohttp
import asyncio
import os
import glob

# Dynamically find all JPEG/PNG images in 'images/' directory
image_dir = 'images'
image_paths = glob.glob(os.path.join(image_dir, '*.[jp][pn]g'))  # matches .jpg and .png files

# For flexibility (or if your images have uppercase extensions), use:
# image_paths = [f for f in glob.glob(os.path.join(image_dir, '*')) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

# 🔧 Limit concurrent requests to avoid overwhelming the server
sem = asyncio.Semaphore(3)  # limit to 5 concurrent requests

async def send_request(session, image_path):
    async with sem:
        try:
            with open(image_path, 'rb') as f:
                 #  Prepare multipart/form-data
                form = aiohttp.FormData()
                form.add_field('file', f, filename=os.path.basename(image_path), content_type='image/jpeg')

                #  Send POST request to /caption endpoint
                async with session.post('http://localhost:80/caption', data=form) as response:
                    if response.status == 200:
                        result = await response.json()
                        print(f'Image: {image_path}, Caption: {result["caption"]}')
                    else:
                        print(f'Error for {image_path}: {response.status}')
        except Exception as e:
            print(f'Error processing {image_path}: {e}')

async def main():
    async with aiohttp.ClientSession() as session:
        #  Create and run parallel tasks
        tasks = [send_request(session, path) for path in image_paths]
        await asyncio.gather(*tasks)

#  In Jupyter, we use 'await' instead of 'asyncio.run()'
await main()


Image: images/bananas.jpg, Caption: a bunch of bananas
Image: images/cycling.jpg, Caption: a man riding a bike
Image: images/elephant.jpg, Caption: the elephant is wet
Error for images/dog.png: 502
Error for images/spoons.jpg: 502
Error for images/bike.jpg: 502
Error for images/surf.jpg: 502
Image: images/cherry_blossom.jpg, Caption: people are gathered around a blue table covered in pink flowers
Image: images/boy_eating_burger.jpg, Caption: a boy eating a piece of pizza
Image: images/family.png, Caption: a group of people are smiling and posing for a picture
